In [1]:
"""Dataset modules for loading HDF5 simulation trajectories."""

import bisect
import json
import os
import os.path as osp
import re
import zipfile
from typing import Callable, Optional

import h5py
import jax.numpy as jnp
import numpy as np
import wget
from torch.utils.data import Dataset

from lagrangebench.utils import NodeType

2024-01-24 12:59:02.434069: W external/xla/xla/service/gpu/nvptx_compiler.cc:679] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.3.103). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [2]:
def get_dataset_name_from_path(path: str) -> str:
    """Infer the dataset name from the provided path.

    This function assumes that the dataset directory name has the following structure:
    {2D|3D}_{TGV|RPF|LDC|DAM}_{num_particles_max}_{num_steps}every{sampling_rate}

    The dataset name then becomes one of the following:
    {tgv2d|tgv3d|rpf2d|rpf3d|ldc2d|ldc3d|dam2d}
    """

    name = re.search(r"(?:2D|3D)_[A-Z]{2}", path)
    assert name is not None, (
        f"No valid dataset name found in path {path}. "
        "Valid name formats: {2D|3D}_{HT|TGV|RPF|LDC|DAM} "
        "Alternatively, you can specify the dataset name explicitly."
    )
    name = name.group(0)
    name = f"{name.split('_')[1]}{name.split('_')[0]}".lower()
    return name

In [3]:
split: str = "train"
dataset_path: str = "datasets/2D_HT_2500_10kevery100"
#name: Optional[str] = None,

nl_backend: str = "jaxmd_vmap",
#external_force_fn: Optional[Callable] = None,

type(dataset_path)


str

In [4]:
#dataset_path = "datasets/2D_HT_2500_10kevery100"
input_seq_length: int =6

extra_seq_length: int =0

#type(dataset_path)

In [5]:
#dataset_path: str = "datasets/2D_HT_2500_10kevery100"
if dataset_path.endswith("/"):  # remove trailing slash in dataset path
            dataset_path = dataset_path[:-1]

dataset_path

'datasets/2D_HT_2500_10kevery100'

In [6]:
name=None

In [7]:
if name is None:
    name = get_dataset_name_from_path(dataset_path)

else:
    name = name

name

'ht2d'

In [8]:
if not osp.exists(dataset_path):
    print('no data')
else:
    print('data available')

data available


In [9]:
assert split in ["train", "valid", "test"]
assert (
            input_seq_length > 1
        )

In [10]:
file_path = osp.join(dataset_path, split + ".h5")
file_path

'datasets/2D_HT_2500_10kevery100/train.h5'

In [11]:
nl_backend: str = "jaxmd_vmap"
external_force_fn: Optional[Callable] = None

In [12]:
with open(osp.join(dataset_path, "metadata.json"), "r") as f:
            metadata = json.loads(f.read())

db_hdf5 = None

#metadata

In [17]:
with h5py.File(file_path, "r") as f:
    traj_keys = list(f.keys())
    sequence_length = f["00000/position"].shape[0]
print(sequence_length)

print(traj_keys)


61
['00000', '00001', '00002', '00003']


In [1]:
if split == "train":
            # During training, the first input_seq_length steps can only be used as
            # input, and the last one to compute the target acceleration. If we use
            # pushforward, then we need to provide extra_seq_length more steps
            # from the end of a trajectory. Thus, the number of training samples per
            # trajectory becomes:
            subseq_length = input_seq_length + 1 + extra_seq_length
            samples_per_traj = sequence_length - subseq_length + 1

            keylens = jnp.array([samples_per_traj for _ in range(len(traj_keys))])
            _keylen_cumulative = jnp.cumsum(keylens).tolist()
            num_samples = sum(keylens)
            getter = get_window()

num_samples

NameError: name 'split' is not defined